# Libs

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.distributions import norm, uniform
from pyDOE import lhs
from tqdm import tqdm
import glob
import psutil
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

pd.set_option.display_precision = 3
pd.set_option.display_float_format = lambda x: '%.5f' % x
pd.set_option.display_max_columns = 15
pd.set_option.display_max_rows = 6

In [ ]:
vm = psutil.virtual_memory()
free_memory = vm.available
free_memory_mb = free_memory / (1024 * 1024)
print(f"Free memory: {free_memory_mb} MB")

# Phit uncertainty

## Phit grain dense

In [3]:
core = pd.read_csv(r'C:\jupyter\SPP_uncertainty\io\core_phit_k.csv')
core.columns = core.columns.str.lower()
core8 = core[core.formation == 'Balakhany VIII']
core8_mtrx = core8[core8.grain_den_h.notna()]
mean = core8_mtrx.grain_den_h.mean().round(2)
stdev_grain_mtrx8 = core8_mtrx.grain_den_h.std().round(3)

## Rhob uncert

In [ ]:
# * grain density range
# * rhob range (+-0.25) - to be confirmed with halliburton
# * mud filtrate range

In [ ]:
mean_noise = 0
sigma_noise = 0.025/2
iteration = 15
noise_random = lhs(1, samples=iteration)
noise_random = norm(loc=mean_noise, scale=sigma_noise).ppf(noise_random).flatten().round(3)
sns.histplot(noise_random, bins=15, kde=True)
print('distr:', noise_random, '\nmax:',noise_random.max(), 'min:',noise_random.min())

In [ ]:
ds_logs = pd.read_csv(r'C:\jupyter\SPP_uncertainty\io\phit8_set_full.csv')
ds_logs = ds_logs.astype({'well':'string','RHOB':'float32','RHOMA':'float32','RHOF':'float32'})
ds_logs = ds_logs[ds_logs.PHIT > 0]
logs_uncert = ds_logs[['well','TST','RHOB','RHOMA','RHOF']]
logs_uncert['rhob_noise'] = 0
df_lst_rhob = []
for idx, var in tqdm(enumerate(noise_random)):
    logs_uncert_copy = logs_uncert.copy()
    logs_uncert_copy['rhob_noise'] = logs_uncert.RHOB + var
    logs_uncert_copy['idx_rhob'] = idx
    df_lst_rhob.append(logs_uncert_copy)
data_rhob = pd.concat(df_lst_rhob, axis=0).reset_index(drop=True)
data_rhob = data_rhob[data_rhob.RHOB > 0].reset_index(drop=True)
data_rhob.to_parquet(r'C:\jupyter\SPP_uncertainty\io\wells\data_rhob.parquet')

## Rhoma uncert

In [ ]:
grain_dense_h_mean = 0
iteration = 15
sigma_mtrx = stdev_grain_mtrx8/2
mean_mtrx = grain_dense_h_mean
lhs_samples = lhs(1, samples=iteration)
mtrx_random = norm(loc=mean_mtrx, scale=sigma_mtrx).ppf(lhs_samples).flatten().round(3)
sns.histplot(mtrx_random, bins=15, kde=True)
print('mean',mean,'stdev',stdev_grain_mtrx8)
print('distr:',mtrx_random, '\nmax:', mtrx_random.max(), 'min:', mtrx_random.min())

In [ ]:
logs_uncert = pd.read_parquet(r'C:\jupyter\SPP_uncertainty\io\wells\data_rhob.parquet')
df_lst_mtrx = []
for idx, var in tqdm(enumerate(mtrx_random)):
    logs_uncert_copy = logs_uncert.copy()
    logs_uncert_copy['rhoma_noise'] = logs_uncert.RHOMA + var
    logs_uncert_copy['idx_rhoma_noise'] = idx
    df_lst_mtrx.append(logs_uncert_copy)
data_mtrx = pd.concat(df_lst_mtrx, axis=0).reset_index(drop=True)
data_mtrx.to_parquet(r'C:\jupyter\SPP_uncertainty\io\wells\data_mtrx.parquet')

## Rhof uncert

In [ ]:
rhof1 = 0.8
print(rhof1, (2.69 - 2.25)/(2.69 - rhof1))
rhof2 = 0.9
print(rhof2, (2.69 - 2.25)/(2.69 - rhof2))
rhof3 = 1
print(rhof3, (2.69 - 2.25)/(2.69 - rhof3))

In [ ]:
rhof_min = 0.83
rhof_max = 0.935
rhof_mean = (rhof_max + rhof_min) / 2
rhof_sigma = (rhof_max - rhof_min) / 4
iteration = 15
fltr_random = lhs(1, samples=iteration)
fltr_random = norm(loc=rhof_mean, scale=rhof_sigma).ppf(fltr_random).flatten().round(3)
sns.histplot(fltr_random, bins=15, kde=True)
print('distr:', fltr_random, '\nmax:',fltr_random.max(), 'min:',fltr_random.min(), 'rhof_sigma:', round(rhof_sigma,3))

In [ ]:
logs_uncert = pd.read_parquet(r'C:\jupyter\SPP_uncertainty\io\wells\data_mtrx.parquet')
logs_uncert.well.unique()
for wellname in tqdm(logs_uncert.well.unique()):
    data = logs_uncert[logs_uncert.well == wellname]
    data_fltrl_lst = []
    for idx, var in (enumerate(fltr_random)):
        data_copy = data.copy()
        data_copy['rhof_noise'] = var
        data_copy['idx_rhof_noise'] = idx
        data_copy['phit_drnd'] = (data_copy['rhoma_noise'] - data_copy['rhob_noise']) / (data_copy['rhoma_noise'] - data_copy['rhof_noise'])
        data_copy = data_copy.drop_duplicates().reset_index(drop=True)
        data_fltrl_lst.append(data_copy)
    data_fltr = pd.concat(data_fltrl_lst, axis=0).reset_index(drop=True)
    data_fltr.to_parquet(r'C:\jupyter\SPP_uncertainty\io\wells\\' + wellname + '.parquet')

## Final computation

### P10,P50 & P90 groupby

In [ ]:
#gas wells with uncertainty problems B01Y, B12, B13ST2, B14Z, B19, B21, B26, B37, C32

file_list = glob.glob(r'C:\jupyter\SPP_uncertainty\io\wells\*.parquet') 
file_list = file_list[:-2]
phit_drnd_lst = []
for file in tqdm(file_list):
    well_one = pd.read_parquet(file)
    p10 = well_one.groupby(['well','TST'])['phit_drnd'].quantile(0.10).reset_index()
    p10['quantile'] = 'p10'
    p50 = well_one.groupby(['well','TST'])['phit_drnd'].quantile(0.50).reset_index()
    p50['quantile'] = 'p50'
    p90 = well_one.groupby(['well','TST'])['phit_drnd'].quantile(0.90).reset_index()
    p90['quantile'] = 'p90'
    data_final = pd.concat([p10, p50, p90], axis=0).reset_index(drop=True)
    data_final['quantile'] = data_final['quantile'].astype('string')
    phit_drnd_lst.append(data_final)
phit_uncertainty = pd.concat(phit_drnd_lst, axis=0).sort_values(by=['well','TST','quantile']).reset_index(drop=True)

ds_logs = pd.read_csv(r'C:\jupyter\SPP_uncertainty\io\phit8_set_full.csv').drop(columns=['Unnamed: 0'], axis=1)
ds_logs = ds_logs[ds_logs.PHIT > 0]
phit_uncertainty_v2 = phit_uncertainty.set_index(['well','TST']).join(ds_logs[['well','TST','PHIT','phit_dn','NET']].set_index(['well','TST'])).reset_index()
phit_uncertainty_v2 = phit_uncertainty_v2.drop_duplicates().reset_index(drop=True)
phit_uncertainty_v2 = phit_uncertainty_v2[phit_uncertainty_v2.phit_drnd < 0.5].reset_index(drop=True)
diff = phit_uncertainty_v2[phit_uncertainty_v2['quantile'] =='p90']['phit_drnd'].values - phit_uncertainty_v2[phit_uncertainty_v2['quantile'] =='p10']['phit_drnd'].values
print('range p90p10 mean', np.mean(diff).round(3))

for wellname in phit_uncertainty_v2.well.unique():
    fig, ax = plt.subplots(figsize=(6, 6))
    phit_uncertainty_well = phit_uncertainty_v2[phit_uncertainty_v2.well == wellname]
    sns.scatterplot(data=phit_uncertainty_well, 
                    x='PHIT', y='phit_drnd', hue='quantile', palette='bright', alpha=0.5, ax=ax)
    sns.lineplot(x=[0.1, 0.35], y=[0.1, 0.35], color='black')
    plt.title(wellname);

In [50]:
phit_rnd_avg = phit_uncertainty_v2.groupby(['well','quantile'])[['phit_drnd', 'PHIT']].mean().reset_index()
phit_rnd_net_avg = phit_uncertainty_v2[phit_uncertainty_v2.NET == 1].groupby(['well','quantile'])[['phit_drnd', 'PHIT']].mean().reset_index()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14,5))
add1 = 0.0155
add2 = 0.005

sns.scatterplot(data=phit_rnd_avg, x='PHIT', y='phit_drnd', hue='quantile', s=50, ax=ax[0])

sns.lineplot(x=[0.12,0.24], y=[0.12,0.24], color='black', linestyle='--', ax=ax[0])
sns.lineplot(x=[0.12,0.24], y=[0.12+add1,0.24+add1], color='red', linestyle='--', ax=ax[0])
sns.lineplot(x=[0.12,0.24], y=[0.12-add2,0.24-add2], color='red', linestyle='--', ax=ax[0])

sns.scatterplot(data=phit_rnd_net_avg, x='PHIT', y='phit_drnd', hue='quantile', s=50, ax=ax[1])
sns.lineplot(x=[0.12,0.28], y=[0.12,0.28], color='black', linestyle='--', ax=ax[1])
sns.lineplot(x=[0.12,0.28], y=[0.12+add1,0.28+add1], color='red', linestyle='--', ax=ax[1])
sns.lineplot(x=[0.12,0.28], y=[0.12-add2,0.28-add2], color='red', linestyle='--', ax=ax[1])
ax[0].set_title('PHIT vs phit_rnd_avg')
ax[1].set_title('PHIT vs phit_rnd_net_avg')
ax[0].grid(True, linestyle='--')
ax[1].grid(True, linestyle='--')

### Logs calculation

In [ ]:
def well_phit_uncert_display(dataset, wellname):
    well_uncrt = dataset[dataset.well == wellname]
    fig, ax = plt.subplots(figsize=(3,10))
    tst = well_uncrt[well_uncrt['quantile'] == 'p50'].TST
    p50 = well_uncrt[well_uncrt['quantile'] == 'p50'].phit_drnd
    p10 = well_uncrt[well_uncrt['quantile'] == 'p10'].phit_drnd
    p90 = well_uncrt[well_uncrt['quantile'] == 'p90'].phit_drnd
    phit_well = well_uncrt[well_uncrt['quantile'] == 'p50'].PHIT
    phit_dn_well = well_uncrt[well_uncrt['quantile'] == 'p50'].phit_dn
    ax.plot(phit_well, tst, label='phit', color='red', ls='-', lw=1, zorder=3)
    ax.plot(phit_dn_well, tst, label='phit_dn', color='gray', ls='-', lw=3, zorder=2)
    ax.plot(p10, tst, label='P10', color='blue', ls='-', lw=1, zorder=2)
    ax.plot(p50, tst, label='test_p50', color='#91091f', ls='-', lw=1, zorder=2)
    ax.plot(p90, tst, label='P90', color='green', ls='-', lw=1, zorder=2)
    ax.invert_yaxis()
    ax.invert_xaxis()
    ax.set_xlim(0.5, 0.0)
    ax.grid(True)
    ax.title.set_text(wellname)
problems_uncert_well_list = ['B01Y', 'B12', 'B13ST2', 'B14Z', 'B19', 'B21', 'B26', 'B37']
for wellname in problems_uncert_well_list:
    well_phit_uncert_display(phit_uncertainty_v2, wellname)

### Sensitivity analysis by LinReg

In [53]:
file_list = glob.glob(r'C:\jupyter\SPP_uncertainty\io\wells\*.parquet')
# file_list

In [ ]:
file_list = glob.glob(r'C:\jupyter\SPP_uncertainty\io\wells\*.parquet')
selected_well = 'B22'
for well_file in file_list:
    wellname = well_file.split('\\')[-1].split('.')[0]
    if selected_well == wellname:
        well_one = pd.read_parquet(well_file)
        break
df_lst = []
for idx_rhob in tqdm(well_one.idx_rhob.unique()):
    for idx_rhoma in well_one[(well_one.idx_rhob == idx_rhob)].idx_rhoma_noise.unique():
        for idx_rhof in well_one[(well_one.idx_rhob == idx_rhob) & (well_one.idx_rhoma_noise == idx_rhoma)].idx_rhof_noise.unique():
            well_one_realization = well_one[(well_one.idx_rhob == idx_rhob) & 
                                            (well_one.idx_rhoma_noise == idx_rhoma) & 
                                            (well_one.idx_rhof_noise == idx_rhof)]
            X = well_one_realization[['rhob_noise','rhoma_noise','rhof_noise']]
            y = well_one_realization['phit_drnd']
            reg = LinearRegression().fit(X, y)
            predictions = reg.predict(X)
            mae = mean_absolute_error(y, predictions)
            coefficients = reg.coef_.round(5)
            df = pd.DataFrame({ 'well':[well_one.well.iloc[0]], 
                                'idx_rhob':[idx_rhob], 
                                'idx_rhoma_noise':[idx_rhoma], 
                                'idx_rhof_noise':[idx_rhof],
                                'mae':[mae],
                                'rhob_noise_coeff':[coefficients[0]],
                                'rhoma_noise_coeff':[coefficients[1]],
                                'rhof_noise_coeff':[coefficients[2]],
                                'rhob_rhoma':[well_one_realization.rhoma_noise.iloc[0]],
                                'rhob_rhof':[well_one_realization.rhof_noise.iloc[0]],})
            df_lst.append(df)
result = pd.concat(df_lst, axis=0).reset_index(drop=True)

In [ ]:
result

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(18, 4))
sns.histplot(result.rhob_noise_coeff, bins=15, kde=True, ax=ax[0], color='red')
sns.histplot(result.rhoma_noise_coeff, bins=15, kde=True, ax=ax[1], color='green')
sns.histplot(result.rhof_noise_coeff, bins=15, kde=True, ax=ax[2], color='#477ae7')
sns.boxplot(result.mae, ax=ax[3], color='orange')
ax[0].set_title(result.well.iloc[0], fontsize=16)
ax[3].set_title('MAE', fontsize=12)
plt.tight_layout();

In [ ]:
bad_case = result[result.rhof_noise_coeff < -0.5]
bad_idx_rhob = bad_case.idx_rhob.iloc[0]
bad_idx_rhoma = bad_case.idx_rhoma_noise.iloc[0]
bad_idx_rhof = bad_case.idx_rhof_noise.iloc[0]

good_case = result[result.rhof_noise_coeff == 0]
good_idx_rhob = good_case.idx_rhob.iloc[135]
good_idx_rhoma = good_case.idx_rhoma_noise.iloc[135]
good_idx_rhof = good_case.idx_rhof_noise.iloc[135]
print('bad_case', bad_idx_rhob, bad_idx_rhoma, bad_idx_rhof)
print('good_case', good_idx_rhob, good_idx_rhoma, good_idx_rhof)
print('bad case')
display(well_one[(well_one.idx_rhob == bad_idx_rhob) & (well_one.idx_rhoma_noise == bad_idx_rhoma) & (well_one.idx_rhof_noise == bad_idx_rhof)].iloc[:1])
print('good case')
display(well_one[(well_one.idx_rhob == good_idx_rhob) & (well_one.idx_rhoma_noise == good_idx_rhoma) & (well_one.idx_rhof_noise == good_idx_rhof)].iloc[:1])